In [1]:
import sys
import os
import torch
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(sys.path[0] + '/../'))

torch.cuda.set_device(2)
print("Set CUDA:%d as current device." % torch.cuda.current_device())
torch.autograd.set_grad_enabled(False)

from data.spherical_view_syn import *
from configs.spherical_view_syn import SphericalViewSynConfig
from utils import netio
from utils import misc
from utils import img
from utils import device
from utils import view
from components.gen_final import GenFinal


def load_net(path):
    config = SphericalViewSynConfig()
    config.from_id(path[:-4])
    config.SAMPLE_PARAMS['perturb_sample'] = False
    # config.print()
    net = config.create_net().to(device.default())
    netio.load(path, net)
    return net


def find_file(prefix):
    for path in os.listdir():
        if path.startswith(prefix):
            return path
    return None


def load_views(data_desc_file) -> view.Trans:
    with open(data_desc_file, 'r', encoding='utf-8') as file:
        data_desc = json.loads(file.read())
        view_centers = torch.tensor(
            data_desc['view_centers'], device=device.default()).view(-1, 3)
        view_rots = torch.tensor(
            data_desc['view_rots'], device=device.default()).view(-1, 3, 3)
        return view.Trans(view_centers, view_rots)


def plot_cross(center, res):
    plt.plot(
        [
            (res[1] - 1) / 2 + center[0] - 5,
            (res[1] - 1) / 2 + center[0] + 5
        ],
        [
            (res[0] - 1) / 2 + center[1],
            (res[0] - 1) / 2 + center[1]
        ],
        color=[0, 1, 0])
    plt.plot(
        [
            (res[1] - 1) / 2 + center[0],
            (res[1] - 1) / 2 + center[0]
        ],
        [
            (res[0] - 1) / 2 + center[1] - 5,
            (res[0] - 1) / 2 + center[1] + 5
        ],
        color=[0, 1, 0])


def plot_fovea(left_images, right_images, left_center, right_center):
    plt.figure(figsize=(8, 4))
    plt.subplot(121)
    img.plot(left_images['fovea'])
    fovea_res = left_images['fovea'].size()[-2:]
    plot_cross((0, 0), fovea_res)
    plt.subplot(122)
    img.plot(right_images['fovea'])
    plot_cross((0, 0), fovea_res)


scenes = {
    'gas': '__0_user_study/us_gas_all_in_one',
    'mc': '__0_user_study/us_mc_all_in_one',
    'bedroom': 'bedroom_all_in_one',
    'gallery': 'gallery_all_in_one',
    'lobby': 'lobby_all_in_one'
}

fov_list = [20, 45, 110]
res_list = [(128, 128), (256, 256), (256, 230)]
res_full = (1600, 1440)


Set CUDA:2 as current device.


In [20]:
centers = {
    'gas': [
        [(3.5, 0), (-3.5, 0)],
        [(1.5, 0), (-1.5, 0)]
    ],
    'mc': [
        [(2, 0), (-2, 0)],
        [(2, 0), (-2, 0)]
    ],
    'bedroom': [
        [(5, 0), (-5, 0)],
        [(6, 0), (-6, 0)],
        [(5, 0), (-5, 0)]
    ],
    'gallery': [
        [(2.5, 0), (-2.5, 0)],
        [(11.5, 0), (-11.5, 0)]
    ]
}
scene = 'bedroom'
os.chdir(sys.path[0] + '/../data/' + scenes[scene])
print('Change working directory to ', os.getcwd())

fovea_net = load_net(find_file('fovea'))
periph_net = load_net(find_file('periph'))

# Load Dataset
views = load_views('demo.json')
print('Dataset loaded.')
print('views:', views.size())
gen = GenFinal(fov_list, res_list, res_full, fovea_net, periph_net,
               device=device.default())

for view_idx in range(views.size()[0]):
    test_view = views.get(view_idx)
    left_images = gen(centers[scene][view_idx][0], view.Trans(
        test_view.trans_point(
            torch.tensor([-0.03, 0, 0], device=device.default())
        ), test_view.r), mono_trans=test_view)
    right_images = gen(centers[scene][view_idx][1], view.Trans(
        test_view.trans_point(
            torch.tensor([0.03, 0, 0], device=device.default())
        ), test_view.r), mono_trans=test_view)
    #plot_fovea(left_images, right_images, centers[scene][view_idx][0],
    #           centers[scene][view_idx][1])
    outputdir = '/home/dengnc/deep_view_syn/data/__2_demo/mono_periph/stereo/'
    misc.create_dir(outputdir)
    # for key in images:
    key = 'blended'
    img.save(left_images[key], '%s%s_view%04d_%s_l.png' % (outputdir, scene, view_idx, key))
    img.save(right_images[key], '%s%s_view%04d_%s_r.png' % (outputdir, scene, view_idx, key))
    stereo_overlap = torch.cat([left_images['blended'][:, 0:1], right_images['blended'][:, 1:3]], dim=1)
    img.save(stereo_overlap, '%s%s_view%04d_%s_stereo.png' % (outputdir, scene, view_idx, key))

    left_images = gen(centers[scene][view_idx][0], view.Trans(
        test_view.trans_point(
            torch.tensor([-0.03, 0, 0], device=device.default())
        ), test_view.r))
    right_images = gen(centers[scene][view_idx][1], view.Trans(
        test_view.trans_point(
            torch.tensor([0.03, 0, 0], device=device.default())
        ), test_view.r))
    #plot_fovea(left_images, right_images, centers[scene][view_idx][0],
    #           centers[scene][view_idx][1])
    outputdir = '/home/dengnc/deep_view_syn/data/__2_demo/stereo/'
    misc.create_dir(outputdir)
    # for key in images:
    key = 'blended'
    img.save(left_images[key], '%s%s_view%04d_%s_l.png' % (outputdir, scene, view_idx, key))
    img.save(right_images[key], '%s%s_view%04d_%s_r.png' % (outputdir, scene, view_idx, key))
    stereo_overlap = torch.cat([left_images['blended'][:, 0:1], right_images['blended'][:, 1:3]], dim=1)
    img.save(stereo_overlap, '%s%s_view%04d_%s_stereo.png' % (outputdir, scene, view_idx, key))


Change working directory to  /home/dengnc/deep_view_syn/data/bedroom_all_in_one
Load net from fovea@nmsl-rgb_e10_fc256x4_d1.00-50.00_s32.pth ...
Load net from periph@nnmsl-rgb_e10_fc64x4_d1.00-50.00_s16.pth ...
Dataset loaded.
views: [3]
